### Challenge 12:  Byte-at-a-time ECB decryption (Simple)

<div class="alert alert-block alert-info">
    
Copy your oracle function to a new function that encrypts buffers under ECB mode using a consistent but unknown key (for instance, assign a single random key, once, to a global variable).

Now take that same function and have it append to the plaintext, BEFORE ENCRYPTING, the following string:

```
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
```
---
    
<div class="alert alert-block alert-warning">
    
### Spoiler alert.
Do not decode this string now. Don't do it.
</div>
    
---
    
Base64 decode the string before appending it. Do not base64 decode the string by hand; make your code do it. The point is that you don't know its contents.

What you have now is a function that produces:

```AES-128-ECB(your-string || unknown-string, random-key)```

It turns out: you can decrypt "unknown-string" with repeated calls to the oracle function!

Here's roughly how:

- Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
- Detect that the function is using ECB. You already know, but do this step anyways.
- Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
- Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
- Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
- Repeat for the next byte.

<div class="alert alert-block alert-warning">
### Congratulations.
This is the first challenge we've given you whose solution will break real crypto. Lots of people know that when you encrypt something in ECB mode, you can see penguins through it. Not so many of them can decrypt the contents of those ciphertexts, and now you can. If our experience is any guideline, this attack will get you code execution in security tests about once a year.

</div>
    
</div>

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
import cryptopals as cp
from numpy.random import randint

In [2]:
# Generate a random key (we won't look at it directly...pretend it's secret)
unknown_key = bytes(list(randint(0, 256, 16)))

In [3]:
def encryption_oracle_2(data, unknown_key):
    """
    Implement this:  
    AES-128-ECB(data || unknown-string, random-key)
    """
    unknown_string = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc\
                      28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZG\
                      J5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5\
                      vLCBJIGp1c3QgZHJvdmUgYnkK"

    unknown_bytes = base64.b64decode(unknown_string)
    AES_input = data + unknown_bytes

    return(AESEncrypt(AES_input, unknown_key))

In [4]:
# Step 1: - Feed identical bytes of your-string to the function 1 at a time 
# --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover 
# the block size of the cipher. You know it, but do this step anyway.

for N_bytes in range(1, 10):
    
    malicious_data = b'A' * N_bytes
    print(f"N_bytes={N_bytes}, len(out)={len(cp.encryption_oracle_2(malicious_data, unknown_key))}")

N_bytes=1, len(out)=144
N_bytes=2, len(out)=144
N_bytes=3, len(out)=144
N_bytes=4, len(out)=144
N_bytes=5, len(out)=144
N_bytes=6, len(out)=160
N_bytes=7, len(out)=160
N_bytes=8, len(out)=160
N_bytes=9, len(out)=160


Length of the output goes up by 16 bytes...we know the block size = 16.

In [5]:
# Step 2:
# - Detect that the function is using ECB. You already know, but do this step anyways.

malicious_data = b'a' * 100
print('\n***********************')
if cp.detect_AES_ECB(cp.encryption_oracle_2(malicious_data, unknown_key)):
    print('ECB Mode WAS Detected')
else:
    print('ECB Mode NOT Detected')
print('***********************\n')


***********************
ECB Mode WAS Detected
***********************



In [6]:
# Step 3-6:
#
# - Knowing the block size, craft an input block that is exactly 1 byte short 
#   (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about 
#   what the oracle function is going to put in that last byte position.
#
# - Make a dictionary of every possible last byte by feeding different strings 
#   to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering 
#   the first block of each invocation.
#
# - Match the output of the one-byte-short input to one of the entries in your 
#   dictionary. You've now discovered the first byte of unknown-string.
#
# - Repeat for the next byte.

plaintext = b''

print('\nProbing the oracle:')

for block_idx in range(1,10):
    print()
    for ii in range(15, -1, -1):
        print('.',end='')
        malicious_data = (b'\x00' * ii)
        output_to_match = cp.encryption_oracle_2(malicious_data, unknown_key)
        malicious_data += plaintext
        
        Found = False
        for chr_idx in range(256):
            # Try a character
            try_data = malicious_data + bytes([chr_idx])
            # print(try_data)
            if cp.encryption_oracle_2(try_data, unknown_key)[0:(block_idx*16)-1] == output_to_match[0:(block_idx*16)-1]:
                Found = True
                # We found this byte...add it to the plaintext.
                plaintext += bytes([chr_idx])
                break
                
        if Found == False:
            print(f"\nNo more matches found.")
            break
print()            
print('*******************')
print('Decoded the secret message!  Plaintext is:\n')
print(plaintext.decode())


Probing the oracle:

................
................
................
................
................
................
................
................
............
No more matches found.

*******************
Decoded the secret message!  Plaintext is:

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)